![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [29]:
import pandas as pd
import numpy as np

# Start coding here...
df = pd.read_csv('bank_marketing.csv')
#pd.set_option('display.max_columns', None)
#print(df.head)

# Split and tidy bank_marketing.csv, storing as three DataFrames called client, campaign, and economics, each containing the columns outlined in the notebook and formatted to the data types listed.

### Split into tables
client = df[["client_id", "age", "job", "marital","education", "credit_default", "mortgage"]]
campaign = df[["client_id", "number_contacts", "month", "day", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
economics = df[["client_id", "cons_price_idx", "euribor_three_months"]]

## Clean client table
### Clean job column. Change "." to "_"
client['job'] = client['job'].str.replace('.','_')

### Clean education column.Change "." to "_"
#### .str.replace(old, new) When you need to modify part of a string
client['education'] = client['education'].str.replace(".","_")

### Replace "unknown" to np.NaN
#### .replace(old, new) When you need to replace entire cell values
client['education'] = client['education'].replace('unknown',np.NaN)

### Clean and convert columns credit_default and mortgage to boolean data type.
### Change credit_default and mortgage to binary values
for i in ["credit_default", "mortgage"]:
    client[i] = client[i].map({'yes':1,
                               'no':0,
                               'unknown':0})
    client[i] = client[i].astype(bool)
    
## Clean campaign table
### Change previous_outcome to binary values: 1 if "success", otherwise 0.
#### Use the pandas Series .map() method to convert values, providing key-value pairs where the key is the current value and the value is what you would like to change it to.
campaign['previous_outcome'] = campaign['previous_outcome'].map({'sucsess':1,
                                                                 'failure':0,
                                                                 'nonexistent':0})
### Change campaign_outcome to binary values: 1 if "yes", otherwise 0.
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'yes':1,
                                                                 'no':0})
### Convert previous_outcome and campaign_outcome to Boolean values
for i in ['previous_outcome', 'campaign_outcome']:
    campaign[i] = campaign[i].astype(bool)

### last_contact_date column is created from a combination of day, month, and a newly created year column (which have a value of 2022); Format = "YYYY-MM-DD"

### Create a year column
campaign['year'] = '2022'

### Convert day to string so we can combine the the columns in the desired format. We dont change month because it's already a string.
campaign['day'] = campaign['day'].astype(str)

### Create a last_contact_date column from a combination of day, month, and year
campaign['last_contact_date'] = campaign['year'] + '-' + campaign['month'] + '-' + campaign['day']

### Convert the column to daytime
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'],format="%Y-%b-%d")

### Drop year, month and day column
#### Only drop() returns a new dataframe without modifying the original.
#### Setting parameter inplace=True modifies the original dataframe directly, therefore we don't need to assign the dataframe back to compaign.
campaign.drop(columns=['year','month','day'], inplace=True)

# Save the three DataFrames to csv files, without an index, as client.csv, campaign.csv, and economics.csv respectively.
#### By default, pandas includes the index as a first column. Seeing parameter index=False, csv is saved without row indices.
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

# print(f"This is client's DataFrame:\n{client.head()}")
# print(f"This is campaign's DataFrame:\n{campaign.head()}")
# print(f"This is economics's DataFrame:\n{economics.head()}")

In [30]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64
